# GPT-2 Transformer Model for Sarcasm Detection

In [1]:
import torch
import numpy as np
import pandas as pd
import os
import json
import pandas as pd
import numpy as np
import nltk
import gensim
import re
import copy
import gc
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from tqdm import tqdm
# https://colab.research.google.com/drive/1dMTdO5vxdVX0NA2Qe7AV9WGEy8ZH67Xn?usp=sharing#scrollTo=afcc233b

/home/lfrostbyte/anaconda3/envs/CS4248/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/lfrostbyte/anaconda3/envs/CS4248/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

print(torch.cuda.is_available())

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/lfrostbyte/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lfrostbyte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lfrostbyte/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True


In [3]:
# train_df = pd.read_json(path_or_buf="./Sarcasm_Headlines_Dataset.json", lines=True)
train_df = pd.read_json(path_or_buf="./Sarcasm_Headlines_Dataset.json", lines=True)
train_df.rename(columns={"headline": "text", "is_sarcastic": "label"}, inplace=True)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word.isalpha()]  # Remove numbers and punctuation
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopword removal
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    
    return tokens if tokens else None

# # For convenience
# train_df = train_df.head(100)

# Apply preprocessing
# train_df['processed_text'] = train_df['text'].apply(preprocess_text)
# train_df = train_df.dropna(subset=['processed_text']) # After preprocessing, there will be some rows that are empty, delete these rows


# Split into train, test, validation (80% train, 20% validation from train)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    train_df['text'], train_df['label'], test_size=0.2, random_state=42)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=42) # Ideally test_size should be 1/8 because 0.8 * 1/8 = 0.1 so its 10% for validation, but 0.08 is also fine.

print(len(train_texts))
print(len(train_labels))

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)
test_texts = test_texts.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

19230
19230


In [4]:
train_texts

0        madonna's anti-trump cover of britney spears' ...
1        chinese scientists successfully clone 2 monkey...
2        john urschel on why kids shouldn't play footba...
3        international criminal court announces new '3 ...
4        pence tells emotional story of longtime friend...
                               ...                        
19225                  darfur, ia also in pretty bad shape
19226    lgbtq activists organizing massive dance prote...
19227    bill nye says empathy is necessary for human s...
19228             white house declares war on dsl provider
19229    marine determined to win heart, mind of at lea...
Name: text, Length: 19230, dtype: object

In [5]:
class GPT2Classifier(torch.nn.Module):
    def __init__(self, hidden_size: int, num_classes: int, max_seq_len: int):
        super(GPT2Classifier,self).__init__()
        self.frontLayer = GPT2Model.from_pretrained('gpt2')
        trf_out_size = hidden_size * max_seq_len
        self.fc = torch.nn.Linear(in_features=trf_out_size, out_features=num_classes)
        self.seq_len = max_seq_len

    def forward(self, input_id, mask):
        """
        Args: input_id: encoded input of ids that were sent
        """
        gpt_out, _ = self.frontLayer(input_ids=input_id, attention_mask=mask, return_dict=False)
        batch_size = gpt_out.shape[0]
        linear_output = self.fc(gpt_out.view(batch_size, -1))
        return linear_output

class SarcasmDataset(Dataset):
    def __init__(self, headlines, labels, tokenizer, max_length=50):
        self.headlines = headlines
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.headlines)
    
    def __getitem__(self, idx):
        processed_headline = self.headlines[idx]
        # processed_headline = " ".join(self.headlines[idx])
        encoded_data = self.tokenizer(processed_headline, padding='max_length', max_length=self.max_length, truncation=True, return_tensors="pt")
        return encoded_data, self.labels[idx]
    
def load_model(path, hidden_size, max_seq_len):
    model = GPT2Classifier(hidden_size=hidden_size, num_classes=2, max_seq_len=max_seq_len)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

def train(model, trainData, valData, lr, max_epochs, early_stop_tol):
    trainLoader = DataLoader(trainData, batch_size=16, shuffle=True)
    valLoader = DataLoader(valData, batch_size=16, shuffle=True)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    did_not_improve_count = 0
    best_val_score = 0
    best_epoch = 0
    model.train()
    
    for epoch_num in range(max_epochs):
        # total_acc_train = 0
        total_loss_train = 0
        train_predictions = []
        train_labels = []
        for encoded_data, train_label in tqdm(trainLoader):
            train_label = train_label.to(device)
            mask = encoded_data['attention_mask'].to(device)
            input_id = encoded_data["input_ids"].squeeze(1).to(device)            

            model.zero_grad()

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()

            # acc = (output.argmax(dim=1) == train_label).sum().item()
            # total_acc_train += acc

            # add original labels
            train_labels += train_label.cpu().numpy().flatten().tolist()
            # get predicitons to list
            train_predictions += output.argmax(dim=1).cpu().numpy().flatten().tolist()

            batch_loss.backward()
            optimizer.step()
        
        # total_acc_val = 0
        total_loss_val = 0
        val_predictions = []
        val_labels = []
        
        with torch.no_grad():
            for val_input, val_label in valLoader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                
                output = model(input_id, mask)
                
                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                # acc = (output.argmax(dim=1)==val_label).sum().item()
                # total_acc_val += acc
                # add original labels
                val_labels += val_label.cpu().numpy().flatten().tolist()
                # get predicitons to list
                val_predictions += output.argmax(dim=1).cpu().numpy().flatten().tolist()
            

        # total_acc_val = total_acc_val/len(valData)
        f1_train = f1_score(train_labels, train_predictions, average='macro')
        f1_val = f1_score(val_labels, val_predictions, average='macro')

        if f1_val > (best_val_score + early_stop_tol):
            best_val_score = f1_val
            did_not_improve_count = 0
            best_epoch = epoch_num
            print(f"Saving new best val acc {best_val_score}")
            torch.save(copy.deepcopy(model.state_dict()), f"./gpt2-synthData-lr{lr}-iter{best_epoch+1}-tol{early_stop_tol}-slen{model.seq_len}.pt")
        else:
            did_not_improve_count += 1

        if did_not_improve_count >= 10:
            break

        print(
            f"Epochs: {epoch_num + 1} | Train Loss: {total_loss_train/len(trainData): .3f} \
            | Train Score: {f1_train: .3f} \
             | Val Loss: {total_loss_val / len(valData): .3f} \
             | Val Score: {f1_val: .3f}")
        
        gc.collect()
        torch.cuda.empty_cache()
        
def evaluate(model, testLoader, length):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    true_labels = []

    total_acc_test = 0
    model.eval()
    with torch.no_grad():
        for test_input, test_label in testLoader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)
            
            output = model(input_id, mask)
                        
            acc = (output.argmax(dim=1)==test_label).sum().item()
            total_acc_test += acc

            # add original labels
            true_labels += test_label.cpu().numpy().flatten().tolist()
            # get predictions to list
            predictions += output.argmax(dim=1).cpu().numpy().flatten().tolist()
    test_score = f1_score(true_labels, predictions, average='macro')
    test_p = precision_score(true_labels, predictions, average='macro')
    test_r = recall_score(true_labels, predictions, average='macro')
    test_acc = total_acc_test / length
    print(f'Test Accuracy: {test_acc: .3f}, F1 Score: {test_score: .3f}, Precision: {test_p}, Recall: {test_r}')
    return true_labels, predictions


In [6]:
# Hyperparameters
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
seq_len = 128
hidden_size = 768
val_tol = 0.01
lr = 1e-5

/home/lfrostbyte/anaconda3/envs/CS4248/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
model = GPT2Classifier(hidden_size=hidden_size, num_classes=2, max_seq_len=seq_len)
trainData = SarcasmDataset(headlines=train_texts, labels=train_labels, tokenizer=tokenizer, max_length=seq_len)
valData = SarcasmDataset(headlines=val_texts, labels=val_labels, tokenizer=tokenizer, max_length=seq_len)

In [8]:
train(model, trainData, valData, lr, 500, val_tol)

100%|██████████| 1283/1283 [05:45<00:00,  3.71it/s]


Saving new best val acc 0.80931297086475
Epochs: 1 | Train Loss:  0.033             | Train Score:  0.729              | Val Loss:  0.026              | Val Score:  0.809


100%|██████████| 1283/1283 [05:50<00:00,  3.66it/s]


Saving new best val acc 0.8382520481912606
Epochs: 2 | Train Loss:  0.023             | Train Score:  0.834              | Val Loss:  0.023              | Val Score:  0.838


100%|██████████| 1283/1283 [06:08<00:00,  3.48it/s]


...

100%|██████████| 1283/1283 [05:48<00:00,  3.68it/s]


Epochs: 16 | Train Loss:  0.020             | Train Score:  0.862              | Val Loss:  0.022              | Val Score:  0.846


100%|██████████| 1283/1283 [05:49<00:00,  3.68it/s]


Epochs: 17 | Train Loss:  0.020             | Train Score:  0.861              | Val Loss:  0.023              | Val Score:  0.836


100%|██████████| 1283/1283 [05:49<00:00,  3.67it/s]


In [ ]:
testData = SarcasmDataset(headlines=test_texts, labels=test_labels, tokenizer=tokenizer, max_length=seq_len)
testLoader = DataLoader(testData, batch_size=16, shuffle=True)

model = load_model("./gpt2-classifier-model-lr1e-05-iter5-tol0.01-slen128.pt", hidden_size=768, max_seq_len=seq_len)

evaluate(model, testLoader, len(testData))

print(len(testData))

Test Accuracy:  0.869, F1 Score:  0.864, Precision: 0.8760050588109114, Recall: 0.8591229055620623
5342


In [9]:
# FOR SYNTHETIC DATASET
# Load dataset (change MyDrive to Shared if it's not the same on yours)
train_df = pd.read_json("synthetic_data_v2.json", lines=True)

# Rename columns if necessary
train_df.rename(columns={"headline": "text", "is_sarcastic": "label"}, inplace=True)

# Separate generated and original
generated_df = train_df[train_df['article_link'] == '']
original_df = train_df[train_df['article_link'] != '']

# Total size
total_size = len(train_df)
train_target_size = int(0.8 * total_size)

# How many original samples are needed to reach 80% with generated included
num_original_needed = train_target_size - len(generated_df)

# Sample required number of original samples for training
orig_train = original_df.sample(n=num_original_needed, random_state=42)

# The rest of original data goes to validation
test = original_df.drop(orig_train.index)

# Combine generated + sampled original for final training set
train = pd.concat([generated_df, orig_train], ignore_index=True)

# Split into texts and labels
train_texts, train_labels = train['text'], train['label']
test_texts, test_labels = test['text'], test['label']

train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_texts = test_texts.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

testData = SarcasmDataset(headlines=test_texts, labels=test_labels, tokenizer=tokenizer, max_length=seq_len)
testLoader = DataLoader(testData, batch_size=16, shuffle=True)

model = load_model("./gpt2-synthData-lr1e-05-iter8-tol0.01-slen128.pt", hidden_size=768, max_seq_len=seq_len)

evaluate(model, testLoader, len(testData))

print(len(testData))


/home/lfrostbyte/anaconda3/envs/CS4248/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Test Accuracy:  0.899, F1 Score:  0.899, Precision: 0.9019334975369457, Recall: 0.8979925847457627
5702
